### **Trading Command Interpreter: Asset Extraction and Timed Execution**

In [ ]:
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
from pprint import pprint

In [ ]:
tools = [
  {
    "type": "function",
    "function": {
      "name": "set_chain",
      "description": "Set chain to trade on",
      "parameters": {
        "type": "object",
        "properties": {
          "chain": {
            "type": "string",
            "description": "Chain to trade on",
          },
        },
        "required": ["chain"],
      },
    }
  },
  {
    "type": "function",
    "function": {
      "name": "set_leverage",
      "description": "Adjust leverage for all available assets",
      "parameters": {
        "type": "object",
        "properties": {
          "leverage": {
            "type": "integer",
            "description": "A number between 1 and 100 specifying the leverage",
          },
        },
        "required": ["leverage"],
      },
    }
  },
  {
    "type": "function",
    "function": {
      "name": "set_default_asset",
      "description": "Set default asset to trade",
      "parameters": {
        "type": "object",
        "properties": {
          "asset": {
            "type": "string"
          },
        },
        "required": ["asset"],
      },
    }
  },
	{
		"type": "function",
		"function": {
		  "name": "set_default_size",
		  "description": "Adjust leverage for all available assets",
		  "parameters": {
        "type": "object",
        "properties": {
          "size": {
            "type": "number"
          }
			},
			"required": ["size"],
		  },
		}
	},
  {
		"type": "function",
		"function": {
		  "name": "set_default_margin",
		  "description": "Set default trading margin",
		  "parameters": {
        "type": "object",
        "properties": {
          "margin": {
            "type": "number"
          }
        },
        "required": ["margin"],
		  },
		}
	},
	{
		"type": "function",
		"function": {
		  "name": "take_profit",
		  "description": "Set a take profit order for a specified asset",
		  "parameters": {
        "type": "object",
        "properties": {
          "size": {
            "type": "integer",
            "description": "A number between 0 and 100 denoting the percentage of the order size",
          },
          "asset": {
            "type": "string",
            "description": "The asset to set the take profit order for",
          },
          "tp": {
            "type": "integer",
            "description": "A number between 0 and 100 denoting the take profit percentage/gain in asset before take profit order is triggered",
          },
        },
        "required": ["size", "asset", "tp"],
		  },
		}
	},
  {
		"type": "function",
		"function": {
		  "name": "stop_loss",
		  "description": "Set a stop loss order for a specified asset",
		  "parameters": {
        "type": "object",
        "properties": {
          "size": {
            "type": "integer",
            "description": "A number between 0 and 100 denoting the percentage of the order size",
          },
          "asset": {
            "type": "string",
            "description": "The asset to set the take profit order for",
          },
          "sl": {
            "type": "integer",
            "description": "A number between 0 and 100 denoting the stop loss percentage/loss in asset before stop loss order is triggered",
          },
        },
        "required": ["size", "asset", "sl"],
		  },
		}
	},
  {
		"type": "function",
		"function": {
		  "name": "buy",
		  "description": "Place a buy order for a specified asset",
		  "parameters": {
        "type": "object",
        "properties": {
          "size": {
            "type": "integer",
            "description": "Order size. Default from config if not specified",
          },
          "asset": {
            "type": "string",
            "description": "Asset to buy. Default from config if not specified",
          },
          "price": {
            "type": "number",
            "description": "Buy price. Market price if unspecified",
          },
          "sl": {
            "type": "number",
            "description": "Stop loss price",
          },
          "tp": {
            "type": "number",
            "description": "Take profit price",
          },
          "chase": {
            "type": "boolean",
            "description": "Whether to chase the limit order until filled",
          },
          "dp": {
            "type": "number",
            "description": "If the chase parameter is true, discard price to stop chasing",
          },
          "tsl": {
            "type": "number",
            "description": "Trailing stop loss. If not set, the trailing stop loss will not be set. ",
          }
        },
        "required": [],
		  },
		}
	},
  {
		"type": "function",
		"function": {
		  "name": "twap_buy",
		  "description": "Divides the total order size by the number of intervals. After the time between intervals, each piece of the divided order will be bought at market price",
		  "parameters": {
        "type": "object",
        "properties": {
          "size": {
            "type": "number",
            "description": "Total order size",
          },
          "asset": {
            "type": "string",
            "description": "Asset to buy",
          },
          "interval": {
              "type": "integer",
              "description": "Time between intervals in minutes"
          },
          "n_interval": {
              "type": "integer",
              "description": "Number of intervals"
          },
        },
        "required": ["size", "asset", "interval_min", "n_interval"],
		  },
		}
	},
  {
		"type": "function",
		"function": {
		  "name": "twap_sell",
		  "description": "Divides the total order size by the number of intervals. After the time between intervals, each piece of the divided order will be sold at market price",
		  "parameters": {
        "type": "object",
        "properties": {
          "size": {
            "type": "number",
            "description": "Total order size",
          },
          "asset": {
            "type": "string",
            "description": "Asset to sell",
          },
          "interval": {
              "type": "integer",
              "description": "Time between intervals in minutes"
          },
          "n_interval": {
              "type": "integer",
              "description": "Number of intervals"
          },
        },
        "required": ["size", "asset", "interval_min", "n_interval"],
		  },
		}
	},
  {
		"type": "function",
		"function": {
		  "name": "pair_buy",
		  "description": "Long one asset while shorting another asset in a pair",
		  "parameters": {
        "type": "object",
        "properties": {
          "size": {
            "type": "number",
            "description": "Is the order size to split between the long and short",
          },
          "pair": {
            "type": "string",
            "description": "Is the pair to long and short. In the format of AssetToLong/AssetToShort",
          },
          "price": {
            "type": "number",
            "description": "Limit price ratio for entering the trade",
          },
          "sl": {
              "type": "number",
              "description": "Stop loss ratio"
          },
          "tp": {
              "type": "number",
              "description": "Take profit ratio"
          },
        },
        "required": ["size", "pair"],
		  },
		}
	},
  {
		"type": "function",
		"function": {
		  "name": "pair_sell",
		  "description": "Short one asset while longing another asset in a pair",
		  "parameters": {
        "type": "object",
        "properties": {
          "size": {
            "type": "number",
            "description": "Is the order size to split between the short and long",
          },
          "pair": {
            "type": "string",
            "description": "Is the pair to long and short. In the format of AssetToShort/AssetToLong",
          },
          "price": {
            "type": "number",
            "description": "Limit price ratio for entering the trade",
          },
          "sl": {
              "type": "number",
              "description": "Stop loss ratio"
          },
          "tp": {
              "type": "number",
              "description": "Take profit ratio"
          },
        },
        "required": ["size", "pair"],
		  },
		}
	},
  {
		"type": "function",
		"function": {
		  "name": "scale_buy",
		  "description": "Scale into a long position by placing limit orders at intervals",
		  "parameters": {
        "type": "object",
        "properties": {
          "total_size": {
            "type": "integer",
            "description": "The size over all intervals",
          },
          "n_intervals": {
            "type": "integer",
            "description": "Number of intervals",
          },
          "asset": {
            "type": "string",
            "description": "The asset to scale buy",
          },
          "lower": {
              "type": "number",
              "description": "The lower price to place orders"
          },
          "upper": {
              "type": "number",
              "description": "The upper price to place orders"
          },
        },
        "required": ["total_size", "n_intervals", "asset", "lower", "upper"],
		  },
		}
	},
  {
		"type": "function",
		"function": {
		  "name": "scale_sell",
		  "description": "Scale into a short position by placing limit orders at intervals",
		  "parameters": {
        "type": "object",
        "properties": {
          "total_size": {
            "type": "integer",
            "description": "The size over all intervals",
          },
          "n_intervals": {
            "type": "integer",
            "description": "Number of intervals",
          },
          "asset": {
            "type": "string",
            "description": "The asset to scale sell",
          },
          "lower": {
              "type": "number",
              "description": "The lower price to place orders"
          },
          "upper": {
              "type": "number",
              "description": "The upper price to place orders"
          },
        },
        "required": ["total_size", "n_intervals", "asset", "lower", "upper"],
		  },
		}
	},
  {
		"type": "function",
		"function": {
		  "name": "view_pnl",
		  "description": "View the current unrealized profit and loss (PnL)",
		  "parameters": {}
		}
	},
  {
		"type": "function",
		"function": {
		  "name": "view_balance",
		  "description": "View current wallet balance",
		  "parameters": {}
		}
	},
  {
		"type": "function",
		"function": {
		  "name": "view_unfilled",
		  "description": "View current unfilled orders",
		  "parameters": {}
		}
	},
  {
		"type": "function",
		"function": {
		  "name": "view_open",
		  "description": "View current open positions",
		  "parameters": {}
		}
	},
  {
    "type": "function",
    "function": {
        "name": "reduce_position",
        "description": "Reduce an open position by a specified percentage",
        "parameters": {
            "type": "object",
            "properties": {
                "asset": {
                    "type": "string",
                    "description": "The asset to reduce the position for",
                },
                "percentage": {
                    "type": "number",
                    "description": "The percentage to reduce the position by",
                }
            },
            "required": ["asset", "percentage"],
        },
    }
}

]

#### 1. Extracting Relevant Assets:

In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')


user_prompt = "If the price of bitcoin is less than ethereum, buy SOL and sell btc. independent of the price of eth and btc buy 3 DAI. if ethereum is less than bitcoin, short ethereum and long btc"



final_msg_content = f"""
List the relevant assets required for execute the following command without further explanation or numbers as bullet points

{user_prompt}
"""



messages = [
    {"role": "system", "content":"Pretend you are an expert trader with knowledge in coding and answer without any further explanation or numbers"},
    {"role": "user", "content": final_msg_content}
    ]
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

pprint(completion.choices[0].message.content)

'- Bitcoin (BTC)\n- Ethereum (ETH)\n- Solana (SOL)\n- DAI'


#### 2. Converting the user prompt(trading command) into a chain of commands, given the relevant assets:

In [ ]:
# first example :Conditional(BTC < ETH)
client = OpenAI(api_key='OPENAI_API_KEY')


user_prompt = "If the price of bitcoin is less than ethereum, buy SOL and sell btc. independent of the price of eth and btc buy 3 DAI. if ethereum is less than bitcoin, short ethereum and long btc"



final_msg_content = f"""
Given the following prices ,
- sol: 2500
- btc: 5000
- eth: 30000
Convert the following prompt into a logical chain of commands concisely as an ordered list based on the information given. Take any conditions into consideration when evaluating which commands should be executed and do not list commands that would not be executed given the information provided.

{user_prompt}
"""



messages = [
    {"role": "system", "content":"Pretend you are an expert trader with knowledge in coding and answer without any further explanation or numbers"},
    {"role": "user", "content": final_msg_content}
    ]
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Print the response content
response = completion.choices[0].message.content

# Format the response to ensure it meets the exact requirements
formatted_response = response.strip()
formatted_response = formatted_response.replace('\n', ', ')

# Add a list numbering prefix
formatted_response = f'1. {formatted_response}'

# Print the formatted response
pprint(formatted_response)

'1. 1. Buy 3 DAI, 2. Buy SOL, 3. Sell BTC'


In [ ]:
#Second Example: Conditional(BTC > ETH)

# Initialize the OpenAI client with your API key
client = OpenAI(api_key='OPENAI_API_KEY')

# Define the user prompt
user_prompt = "If the price of bitcoin is less than ethereum, buy SOL and sell BTC. Independent of the price of ETH and BTC, buy 3 DAI. If Ethereum is less than Bitcoin, short Ethereum and long BTC."

# Update the final message content for a single-line, comma-separated response
final_msg_content = f"""
Given the following prices:

- SOL: 2500
- BTC: 50000
- ETH: 30000

Convert the following prompt into a concise, single-line, comma-separated list of all commands to be executed based on the conditions provided. Ensure to list only the commands that will be executed.

{user_prompt}
"""

# Define the messages for the OpenAI model
messages = [
    {"role": "system", "content": "You are an expert trader and need to format the commands as a single-line list with each command separated by a comma. Do not include any explanations or additional text."},
    {"role": "user", "content": final_msg_content}
]

# Get the completion from OpenAI model
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    temperature=0,
)

# Print the response content
response = completion.choices[0].message.content

# Format the response to ensure it meets the exact requirements
formatted_response = response.strip()
formatted_response = formatted_response.replace('\n', ', ')

# Add a list numbering prefix
formatted_response = f'1. {formatted_response}'

# Print the formatted response
pprint(formatted_response)

'1. buy 3 DAI, short Ethereum, long BTC'


### Testing the function calls on different user prompts(trading commands), using the above method:

#### First example for testing:

In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = """
'1. buy 3 DAI, short Ethereum, long BTC'
"""

final_msg_content = f"""
Given the following prices,

- sol: 2500
- btc: 50000
- eth: 30000

execute the following command based on the information given:

{user_prompt}
"""

messages = [
    {"role": "system", "content":"You are a trading assistant. Interpret and execute all the given trading commands based on given conditions."},
    {"role": "user", "content": final_msg_content}
    ]
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  tools=tools,
  tool_choice="auto",
  # parallel_tool_calls=False
)

pprint(completion.choices)

[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_rfANJFEsXUJ6ZIMyjtSeOrzt', function=Function(arguments='{"size": 3, "asset": "DAI"}', name='buy'), type='function'), ChatCompletionMessageToolCall(id='call_tIlXhaKa8WZMHoXhPi0wOrnx', function=Function(arguments='{"size": 3, "pair": "ETH/BTC"}', name='pair_sell'), type='function')]))]


#### Second example for testing:

In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')


user_prompt = "If the current wallet balance exceeds 15,000, set the leverage to 70, the default asset to BTC, and the default margin to 2,000."


final_msg_content = f"""
List the relevant assets required for execute the following command without further explanation or numbers as bullet points

{user_prompt}
"""



messages = [
    {"role": "system", "content":"Pretend you are an expert trader with knowledge in coding and answer without any further explanation or numbers"},
    {"role": "user", "content": final_msg_content}
    ]
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

pprint(completion.choices[0].message.content)

('- Current wallet balance\n'
 '- Leverage setting\n'
 '- Default asset\n'
 '- Default margin')


In [ ]:
from openai import OpenAI
from pprint import pprint

# Initialize the OpenAI client with your API key
client = OpenAI(api_key='OPENAI_API_KEY')

# Define the user prompt
user_prompt = "If the current wallet balance exceeds 15,000, set the leverage to 70, the default asset to BTC, and the default margin to 2,000."
# Update the final message content for a single-line, comma-separated response
final_msg_content = f"""
Given the following prices and assets,
- Current wallet balance: 20000
- Leverage setting: 40
- Default asset: ETH
- Default margin: 1200

Convert the following prompt into a concise, single-line, comma-separated list of all commands to be executed based on the conditions provided. Ensure to list only the commands that will be executed.

{user_prompt}
"""

# Define the messages for the OpenAI model
messages = [
    {"role": "system", "content": "You are an expert trader and need to format the commands as a single-line list with each command separated by a comma. Do not include any explanations or additional text."},
    {"role": "user", "content": final_msg_content}
]

# Get the completion from OpenAI model
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    temperature=0,
)

# Print the response content
response = completion.choices[0].message.content

# Format the response to ensure it meets the exact requirements
formatted_response = response.strip()
formatted_response = formatted_response.replace('\n', ', ')

# Add a list numbering prefix
formatted_response = f'1. {formatted_response}'

# Print the formatted response
pprint(formatted_response)

'1. set leverage 70, set default asset BTC, set default margin 2000'


In [ ]:
from openai import OpenAI
from pprint import pprint

client = OpenAI(api_key='OPENAI_API_KEY')



user_prompt = """
'1. set leverage 70, set default asset BTC, set default margin 2000'
"""

final_msg_content = f"""
Given the following prices and assets,
- Current wallet balance: 20000
- Leverage setting: 40
- Default asset: ETH
- Default margin: 1200

execute the following command based on the information given:

{user_prompt}
"""

messages = [
    {"role": "system", "content":"You are a trading assistant. Interpret and execute all the given trading commands based on given conditions."},
    {"role": "user", "content": final_msg_content}
    ]
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  tools=tools,
  tool_choice="auto",
  # parallel_tool_calls=False
)

pprint(completion.choices)

[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_N9636SNiN4FeEtt4IPnkdgdG', function=Function(arguments='{"leverage": 70}', name='set_leverage'), type='function'), ChatCompletionMessageToolCall(id='call_6WmouWMIFKsn3w2CVDn9ZaDh', function=Function(arguments='{"asset": "BTC"}', name='set_default_asset'), type='function'), ChatCompletionMessageToolCall(id='call_zimSlTguKY7DDX7NeuHpUcP6', function=Function(arguments='{"margin": 2000}', name='set_default_margin'), type='function')]))]


#### Third example for testing:

In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')


user_prompt = "If my portfolio balance drops by 10%, reduce all open positions by 50%."



final_msg_content = f"""
List the relevant assets required for execute the following command without further explanation or numbers as bullet points

{user_prompt}
"""



messages = [
    {"role": "system", "content":"Pretend you are an expert trader with knowledge in coding and answer without any further explanation or numbers"},
    {"role": "user", "content": final_msg_content}
    ]
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

pprint(completion.choices[0].message.content)

('- Portfolio balance data\n'
 '- Open positions data\n'
 '- Trading platform API access\n'
 '- Scripting or automation tool\n'
 '- Risk management rules\n'
 '- Market data feed\n'
 '- Authentication credentials')


In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')


user_prompt = "If my portfolio balance drops by 10%, reduce all open positions by 50%."



final_msg_content = f"""
List just the relevant assets required for execute the following command without further explanation or numbers as bullet points

{user_prompt}
"""



messages = [
    {"role": "system", "content":"Pretend you are an expert trader with knowledge in coding and answer without any further explanation or numbers"},
    {"role": "user", "content": final_msg_content}
    ]
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

pprint(completion.choices[0].message.content)

'- Portfolio balance\n- Open positions'


In [ ]:
# Initialize the OpenAI client with your API key
client = OpenAI(api_key='OPENAI_API_KEY')

# Define the user prompt
user_prompt =  "If my portfolio balance drops by 10%, reduce all open positions by 50%."
# Update the final message content for a single-line, comma-separated response
final_msg_content = f"""
Given the following prices and assets,
- Portfolio Balance Data:
    Initial Balance: $100,000
    Current Balance: $90,000

- Open Positions Data:
    Position 1:
        Asset: Bitcoin (BTC)
        Size: 1 BTC
        Entry Price: $50,000
        Current Price: $45,000
    Position 2:
        Asset: Ethereum (ETH)
        Size: 10 ETH
        Entry Price: $3,000
        Current Price: $2,700

Convert the following prompt into a concise, single-line, comma-separated list of all commands to be executed based on the conditions provided. Ensure to list only the commands that will be executed.

{user_prompt}
"""

# Define the messages for the OpenAI model
messages = [
    {"role": "system", "content": "You are an expert trader and need to format the commands as a single-line list with each command separated by a comma. Do not include any explanations or additional text."},
    {"role": "user", "content": final_msg_content}
]

# Get the completion from OpenAI model
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    temperature=0,
)

# Print the response content
response = completion.choices[0].message.content

# Format the response to ensure it meets the exact requirements
formatted_response = response.strip()
formatted_response = formatted_response.replace('\n', ', ')

# Add a list numbering prefix
formatted_response = f'1. {formatted_response}'

# Print the formatted response
pprint(formatted_response)

'1. Reduce Position 1 by 50%, Reduce Position 2 by 50%'


In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')


user_prompt = """
'1. Reduce Position 1 by 50%, Reduce Position 2 by 50%'
"""

final_msg_content = f"""
Given the following prices and assets,
- Portfolio Balance Data:
    Initial Balance: $100,000
    Current Balance: $90,000

- Open Positions Data:
    Position 1:
        Asset: Bitcoin (BTC)
        Size: 1 BTC
        Entry Price: $50,000
        Current Price: $45,000
    Position 2:
        Asset: Ethereum (ETH)
        Size: 10 ETH
        Entry Price: $3,000
        Current Price: $2,700

execute the following command based on the information given:

{user_prompt}
"""

messages = [
    {"role": "system", "content":"You are a trading assistant. Interpret and execute all the given trading commands based on given conditions."},
    {"role": "user", "content": final_msg_content}
    ]
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  tools=tools,
  tool_choice="auto",
  # parallel_tool_calls=False
)

pprint(completion.choices)

[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ZB98FSJZKR0tb67h5eX8bP3d', function=Function(arguments='{"asset": "Bitcoin", "percentage": 50}', name='reduce_position'), type='function'), ChatCompletionMessageToolCall(id='call_6dI33pKExzLp2Dyl09vKUbCZ', function=Function(arguments='{"asset": "Ethereum", "percentage": 50}', name='reduce_position'), type='function')]))]


#### Fourth example for testing:

In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')


user_prompt = "If SOL's price drops below 2,000, buy 10 units of SOL and set a take profit order for SOL at 2,200 with a 20% gain."



final_msg_content = f"""
List the relevant assets required for execute the following command without further explanation or numbers as bullet points

{user_prompt}
"""



messages = [
    {"role": "system", "content":"Pretend you are an expert trader with knowledge in coding and answer without any further explanation or numbers"},
    {"role": "user", "content": final_msg_content}
    ]
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

pprint(completion.choices[0].message.content)

('- SOL (Solana)\n'
 '- USD (or equivalent stablecoin)\n'
 '- Trading platform or exchange account\n'
 '- API access or trading interface\n'
 '- Conditional order functionality\n'
 '- Take profit order functionality')


In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')


user_prompt = "If SOL's price drops below 2,000, buy 10 units of SOL and set a take profit order for SOL at 2,200 with a 20% gain."



final_msg_content = f"""
List just the relevant assets required for execute the following command without further explanation or numbers as bullet points

{user_prompt}
"""



messages = [
    {"role": "system", "content":"Pretend you are an expert trader with knowledge in coding and answer without any further explanation or numbers"},
    {"role": "user", "content": final_msg_content}
    ]
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

pprint(completion.choices[0].message.content)

'- SOL (Solana)\n- USD (or the base currency used for the transaction)'


In [ ]:
# Initialize the OpenAI client with your API key
client = OpenAI(api_key='OPENAI_API_KEY')

# Define the user prompt
user_prompt =  "If SOL's price drops below 2,000, buy 10 units of SOL and set a take profit order for SOL at 2,200 with a 20% gain."
# Update the final message content for a single-line, comma-separated response
final_msg_content = f"""
Given the following prices and assets,
- SOL : 1950
Convert the following prompt into a concise, single-line, comma-separated list of all commands to be executed based on the conditions provided. Ensure to list only the commands that will be executed.

{user_prompt}
"""

# Define the messages for the OpenAI model
messages = [
    {"role": "system", "content": "You are an expert trader and need to format the commands as a single-line list with each command separated by a comma. Do not include any explanations or additional text."},
    {"role": "user", "content": final_msg_content}
]

# Get the completion from OpenAI model
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    temperature=0,
)

# Print the response content
response = completion.choices[0].message.content

# Format the response to ensure it meets the exact requirements
formatted_response = response.strip()
formatted_response = formatted_response.replace('\n', ', ')

# Add a list numbering prefix
formatted_response = f'1. {formatted_response}'

# Print the formatted response
pprint(formatted_response)

'1. buy 10 units of SOL, set take profit order for SOL at 2200'


In [ ]:
from openai import OpenAI
from pprint import pprint

client = OpenAI(api_key='OPENAI_API_KEYn')



user_prompt = """
'1. buy 10 units of SOL, set take profit order for SOL at 2200'
"""

final_msg_content = f"""
Given the following prices and assets,
- SOL : 1950
execute the following command based on the information given:

{user_prompt}
"""

messages = [
    {"role": "system", "content":"You are a trading assistant. Interpret and execute all the given trading commands based on given conditions."},
    {"role": "user", "content": final_msg_content}
    ]
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  tools=tools,
  tool_choice="auto",
  # parallel_tool_calls=False
)

pprint(completion.choices)

[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_vcsQlIm3oqG5jTxvplSdwPWL', function=Function(arguments='{"size": 10, "asset": "SOL"}', name='buy'), type='function'), ChatCompletionMessageToolCall(id='call_ENzxL04jZBuBW5JkVbKgNtm3', function=Function(arguments='{"size": 10, "asset": "SOL", "tp": 2200}', name='take_profit'), type='function')]))]


#### Fifth example for testing:

In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')


user_prompt = "If BTC's price decreases by more than 7% from the last 24-hour high, buy 10 units of BTC and set a trailing stop loss at 5% below the purchase price."



final_msg_content = f"""
List the relevant assets required for execute the following command without further explanation or numbers as bullet points

{user_prompt}
"""



messages = [
    {"role": "system", "content":"Pretend you are an expert trader with knowledge in coding and answer without any further explanation or numbers"},
    {"role": "user", "content": final_msg_content}
    ]
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

pprint(completion.choices[0].message.content)

('- BTC (Bitcoin)\n'
 '- 24-hour high price data\n'
 '- Current BTC price\n'
 '- Purchase order mechanism\n'
 '- Trailing stop loss mechanism\n'
 '- Percentage change calculation function\n'
 '- Trading platform or exchange API\n'
 '- Sufficient funds for purchase')


In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')


user_prompt = "If BTC's price decreases by more than 7% from the last 24-hour high, buy 10 units of BTC and set a trailing stop loss at 5% below the purchase price."



final_msg_content = f"""
List just the relevant assets required for execute the following command without further explanation or numbers as bullet points

{user_prompt}
"""



messages = [
    {"role": "system", "content":"Pretend you are an expert trader with knowledge in coding and answer without any further explanation or numbers"},
    {"role": "user", "content": final_msg_content}
    ]
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

pprint(completion.choices[0].message.content)

'- BTC (Bitcoin)\n- USD (or the currency used for the purchase)'


In [ ]:
# Initialize the OpenAI client with your API key
client = OpenAI(api_key='OPENAI_API_KEY')

# Define the user prompt
user_prompt = "If BTC's price decreases by more than 7% from the last 24-hour high, buy 10 units of BTC and set a trailing stop loss at 5% below the purchase price."
# Update the final message content for a single-line, comma-separated response
final_msg_content = f"""
Given the following prices and assets,
- 24-Hour High Price of BTC: $50,000
- BTC Current Price : $46,000
- Purchase Details:
    Purchase Price of BTC: $46,000 per unit
    Units of BTC to Buy: 10
Convert the following prompt into a concise, single-line, comma-separated list of all commands to be executed based on the conditions provided. Ensure to list only the commands that will be executed.

{user_prompt}
"""

# Define the messages for the OpenAI model
messages = [
    {"role": "system", "content": "You are an expert trader and need to format the commands as a single-line list with each command separated by a comma. Do not include any explanations or additional text."},
    {"role": "user", "content": final_msg_content}
]

# Get the completion from OpenAI model
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    temperature=0,
)

# Print the response content
response = completion.choices[0].message.content

# Format the response to ensure it meets the exact requirements
formatted_response = response.strip()
formatted_response = formatted_response.replace('\n', ', ')

# Add a list numbering prefix
formatted_response = f'1. {formatted_response}'

# Print the formatted response
pprint(formatted_response)

'1. Buy 10 units of BTC at $46,000, Set trailing stop loss at 5% below $46,000'


In [ ]:
from openai import OpenAI
from pprint import pprint

client = OpenAI(api_key='OPENAI_API_KEY')



user_prompt = """
'1. Buy 10 units of BTC at $46,000, Set trailing stop loss at 5% below $46,000'
"""

final_msg_content = f"""
Given the following prices and assets,
- 24-Hour High Price of BTC: $50,000
- BTC Current Price : $46,000
- Purchase Details:
    Purchase Price of BTC: $46,000 per unit
    Units of BTC to Buy: 10
execute the following command based on the information given:

{user_prompt}
"""

messages = [
    {"role": "system", "content":"You are a trading assistant. Interpret and execute all the given trading commands based on given conditions."},
    {"role": "user", "content": final_msg_content}
    ]
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  tools=tools,
  tool_choice="auto",
  # parallel_tool_calls=False
)

pprint(completion.choices)

[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_90awomIixreaeRSfRB8vQeeh', function=Function(arguments='{\n  "size": 10,\n  "asset": "BTC",\n  "price": 46000,\n  "tsl": 5\n}', name='buy'), type='function')]))]


#### 6th example for testing:

In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')


user_prompt = "If ETH's price is between 30,000 and 35,000, place a scale buy order for ETH with a total size of 6 units over 3 intervals, with prices ranging from 30,000 to 33,000."


final_msg_content = f"""
List just the relevant assets required for execute the following command without further explanation or numbers as bullet points.

{user_prompt}
"""



messages = [
    {"role": "system", "content":"Pretend you are an expert trader with knowledge in coding and answer without any further explanation or numbers"},
    {"role": "user", "content": final_msg_content}
    ]
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

pprint(completion.choices[0].message.content)

'- ETH\n- USD'


In [ ]:
# Initialize the OpenAI client with your API key
client = OpenAI(api_key='OPENAI_API_KEY')

# Define the user prompt
user_prompt = "If ETH's price is between 30,000 and 35,000, place a scale buy order for ETH with a total size of 6 units over 3 intervals, with prices ranging from 30,000 to 33,000."
# Update the final message content for a single-line, comma-separated response
final_msg_content = f"""
Given the following prices and assets,
- ETH: 31000
Convert the following prompt into a concise, single-line, comma-separated list of all commands to be executed based on the conditions provided. Ensure to list only the commands that will be executed.

{user_prompt}
"""

# Define the messages for the OpenAI model
messages = [
    {"role": "system", "content": "You are an expert trader and need to format the commands as a single-line list with each command separated by a comma. Do not include any explanations or additional text."},
    {"role": "user", "content": final_msg_content}
]

# Get the completion from OpenAI model
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    temperature=0,
)

# Print the response content
response = completion.choices[0].message.content

# Format the response to ensure it meets the exact requirements
formatted_response = response.strip()
formatted_response = formatted_response.replace('\n', ', ')

# Add a list numbering prefix
formatted_response = f'1. {formatted_response}'

# Print the formatted response
pprint(formatted_response)

'1. scale_buy_order ETH 6 3 30000 33000'


In [ ]:
from openai import OpenAI
from pprint import pprint

client = OpenAI(api_key='OPENAI_API_KEY')



user_prompt = """
'1. scale_buy_order ETH 6 3 30000 33000'
"""

final_msg_content = f"""
Given the following prices and assets,
- 24-Hour High Price of BTC: $50,000
- BTC Current Price : $46,000
- Purchase Details:
    Purchase Price of BTC: $46,000 per unit
    Units of BTC to Buy: 10
execute the following command based on the information given:

{user_prompt}
"""

messages = [
    {"role": "system", "content":"You are a trading assistant. Interpret and execute all the given trading commands based on given conditions."},
    {"role": "user", "content": final_msg_content}
    ]
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  tools=tools,
  tool_choice="auto",
  # parallel_tool_calls=False
)

pprint(completion.choices)

[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_aIBipljG3MjeSJx2wWILoAQM', function=Function(arguments='{"total_size":6,"n_intervals":3,"asset":"ETH","lower":30000,"upper":33000}', name='scale_buy'), type='function')]))]


## **Trading commands including time**

#### **First method for interpreting trading commands including time and executing their corresponding API calls**
(without getting the list of required times,  given just the current time):




#### First example:

In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

# Define the user prompt
user_prompt = "Buy 10 ETH after 15 minutes and sell 5 ADA after 30 minutes."

# Update the final message content for a specific timing response
final_msg_content = f"""
Given the following command, list the actions and their specific timing without further explanation:

{user_prompt}
"""

# Define the messages for the OpenAI model
messages = [
    {"role": "system", "content": "You are an expert trader and need to format the commands as a list with specific timing for each action. Do not include any explanations or additional text."},
    {"role": "user", "content": final_msg_content}
]

# Get the completion from OpenAI model
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    temperature=0,
)

# Print the response content
response = completion.choices[0].message.content

# Format the response to ensure it meets the exact requirements
formatted_response = response.strip()
formatted_response = formatted_response.replace('\n', ', ')

# Print the formatted response
pprint(formatted_response)

'1. Buy 10 ETH after 15 minutes, 2. Sell 5 ADA after 30 minutes'


In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')


user_prompt = """
'1. Buy 10 ETH after 15 minutes, 2. Sell 5 ADA after 30 minutes'
"""

final_msg_content = f"""
Given the following time,
- current time : 10 AM
execute the following commands at their specified times:

{user_prompt}
"""

messages = [
    {"role": "system", "content":"You are a trading assistant. Interpret and execute all the given trading commands at their specified times."},
    {"role": "user", "content": final_msg_content}
    ]
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  tools=tools,
  tool_choice="auto",
  # parallel_tool_calls=False
)

pprint(completion.choices)


[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_3iWvJncZZcqrGcTx05sydph2', function=Function(arguments='{"size": 10, "asset": "ETH"}', name='buy'), type='function'), ChatCompletionMessageToolCall(id='call_60Dvp1HxTZEvKVaUev20Eh0q', function=Function(arguments='{"size": 5, "asset": "ADA", "n_interval": 1}', name='twap_sell'), type='function')]))]


In [ ]:
# Initialize the OpenAI client with your API key
client = OpenAI(api_key='OPENAI_API_KEY')

# Define the user prompt
user_prompt = """
'1. Buy 10 ETH after 15 minutes, 2. Sell 5 ADA after 30 minutes'
"""

# Define the final message content
final_msg_content = f"""
Given the following time,
- current time: 10 AM
execute the following commands at their specified times:

{user_prompt}
"""

# Define the messages for the OpenAI model
messages = [
    {
        "role": "system",
        "content": """
        1. You are a trading assistant. Interpret and execute all the given trading commands at their specified times.
        2. Provide the exact time each function should be executed in the response.
        """
    },
    {"role": "user", "content": final_msg_content}
]

# Get the completion from OpenAI model
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    tool_choice="auto",
    temperature=0,
)

assistant_message = completion.choices[0].message
messages.append(assistant_message)
assistant_message

ChatCompletionMessage(content='The commands will be executed as follows:\n\n1. Buy 10 ETH at 10:15 AM.\n2. Sell 5 ADA at 10:30 AM.\n\nI will schedule these commands accordingly.\n\n### Schedule for 10:15 AM\n```json\n{\n  "recipient_name": "functions.buy",\n  "parameters": {\n    "size": 10,\n    "asset": "ETH"\n  }\n}\n```\n\n### Schedule for 10:30 AM\n```json\n{\n  "recipient_name": "functions.twap_sell",\n  "parameters": {\n    "size": 5,\n    "asset": "ADA",\n    "n_interval": 1\n  }\n}\n```\n\nI will now proceed to execute these commands at their specified times.', role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_pNtrLhRo5XZBT06lLtGJtfhb', function=Function(arguments='{"size": 10, "asset": "ETH"}', name='buy'), type='function'), ChatCompletionMessageToolCall(id='call_x3FgmJnxy9ocj2u8LH7WUolx', function=Function(arguments='{"size": 5, "asset": "ADA", "n_interval": 1}', name='twap_sell'), type='function')])

#### Second example:

In [ ]:
# Initialize the OpenAI client with your API key
client = OpenAI(api_key='OPENAI_API_KEY')

# Define the user prompt
user_prompt = "Buy 100 XRP now, after 10 minutes set a take profit order for XRP, and after 25 minutes sell 10 XRP."

# Update the final message content for a specific timing response
final_msg_content = f"""
Given the following command, list the actions and their specific timing in the following format:
'first determine the action then its corresponded specified time'. Do not include any explanations or additional text.

{user_prompt}
"""

# Define the messages for the OpenAI model
messages = [
    {"role": "system", "content": "You are an expert trader and need to format the commands in the form 'first determine the action then its corresponded specified time' for each action. Do not include any explanations or additional text."},
    {"role": "user", "content": final_msg_content}
]

# Get the completion from OpenAI model
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    temperature=0,
)

# Print the response content
response = completion.choices[0].message.content

# Format the response to ensure it meets the exact requirements
formatted_response = response.strip()

# Print the formatted response
pprint(formatted_response)

('Buy 100 XRP now\n'
 'Set a take profit order for XRP after 10 minutes\n'
 'Sell 10 XRP after 25 minutes')


In [ ]:
# Initialize the OpenAI client with your API key
client = OpenAI(api_key='OPENAI_API_KEY')

# Define the user prompt
user_prompt = """
'Buy 100 XRP now\n'
 'Set a take profit order for XRP after 10 minutes\n'
 'Sell 10 XRP after 25 minutes'
"""

# Define the final message content
final_msg_content = f"""
Given the following time and asset ,
- current time: 10 AM
execute all the following commands at their specified times:

{user_prompt}
"""

# Define the messages for the OpenAI model
messages = [
    {
        "role": "system",
        "content": """
        1. You are a trading assistant. Interpret and execute all the given trading commands.
        2. Provide the time for all the functions that should be executed at specified time in the response too.
        """
    },
    {"role": "user", "content": final_msg_content}
]

# Get the completion from OpenAI model
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    tool_choice="auto",
    temperature=0,
)

assistant_message = completion.choices[0].message
messages.append(assistant_message)
assistant_message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_OIR2JIVNCIeYWHJhbsobIO96', function=Function(arguments='{"size": 100, "asset": "XRP"}', name='buy'), type='function'), ChatCompletionMessageToolCall(id='call_a1ZaNgC1L4hLtJ2MbKtsawt4', function=Function(arguments='{"size": 100, "asset": "XRP", "tp": 10}', name='take_profit'), type='function'), ChatCompletionMessageToolCall(id='call_HSrGXNmmTs2dzGwYPeHfeU8v', function=Function(arguments='{"size": 10, "asset": "XRP", "interval": 25, "n_interval": 1}', name='twap_sell'), type='function')])

#### **Second method for interpreting trading commands including time and executing their corresponding API calls**
(getting the list of all required times, given just the current time):




#### First example:

In [ ]:
# Initialize the OpenAI client with your API key
client = OpenAI(api_key='OPENAI_API_KEY')

# Define the user prompt
user_prompt = """
Buy 100 XRP now, after 10 minutes set a take profit order for XRP, and after 25 minutes sell 10 XRP.
"""

# Define the final message content
final_msg_content = f"""
List the relevant times required to execute the following command without further explanation or numbers as bullet points:

{user_prompt}
"""

# Define the messages for the OpenAI model
messages = [
     {"role": "system", "content":"Pretend you are an expert trader with knowledge in coding and answer without any further explanation or numbers"},
    {"role": "user", "content": final_msg_content}
    ]

# Get the completion from OpenAI model
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    temperature=0,
)

pprint(completion.choices[0].message.content)


'- Current time (T0)\n- T0 + 10 minutes\n- T0 + 25 minutes'


In [ ]:
# Initialize the OpenAI client with your API key
client = OpenAI(api_key='OPENAI_API_KEY')

# Define the user prompt
user_prompt = "Buy 100 XRP now, after 10 minutes set a take profit order for XRP, and after 25 minutes sell 10 XRP."

# Update the final message content for a specific timing response
final_msg_content = f"""
Given the following time,
- current time : 10 AM ,
 Convert the following prompt into a concise, single-line, comma-separated list of actions and their specific timing in the following format: 'first determine the action then its corresponded specified time'. Do not include any explanations or additional text.

{user_prompt}
"""

# Define the messages for the OpenAI model
messages = [
    {"role": "system", "content": "You are an expert trader and need to format the commands in the form 'first determine the action then its corresponded specified time' for each action. Do not include any explanations or additional text."},
    {"role": "user", "content": final_msg_content}
]

messages = [
    {"role": "system", "content": "You are an expert trader and need to format the commands as a single-line list with each command separated by a comma in the form 'first determine the action then its corresponded specified time' for each action. Do not include any explanations or additional text."},
    {"role": "user", "content": final_msg_content}
]


# Get the completion from OpenAI model
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    temperature=0,
)

# Print the response content
response = completion.choices[0].message.content

# Format the response to ensure it meets the exact requirements
formatted_response = response.strip()
formatted_response = formatted_response.replace('\n', ', ')

# Add a list numbering prefix
formatted_response = f'1. {formatted_response}'

# Print the formatted response
pprint(formatted_response)

('1. Buy 100 XRP 10 AM, set a take profit order for XRP 10:10 AM, sell 10 XRP '
 '10:25 AM')


In [ ]:
# Initialize the OpenAI client with your API key
client = OpenAI(api_key='OPENAI_API_KEY')

# Define the user prompt
user_prompt = """
'1. Buy 100 XRP 10 AM, set a take profit order for XRP 10:10 AM.
"""

# Define the final message content
final_msg_content = f"""
Given the following time and asset ,
- current time: 10 AM
execute all the following commands:

{user_prompt}
"""

# Define the messages for the OpenAI model
messages = [
    {
        "role": "system",
        "content": """
        You are a trading assistant. Interpret and execute all the given trading commands,
        """
    },
    {"role": "user", "content": final_msg_content}
]

# Get the completion from OpenAI model
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    tool_choice="auto",
    temperature=0,
)

assistant_message = completion.choices[0].message
messages.append(assistant_message)
assistant_message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_wMPoHfPq3GETxLrnmM7owhAR', function=Function(arguments='{"size": 100, "asset": "XRP"}', name='buy'), type='function'), ChatCompletionMessageToolCall(id='call_NOuSj6P6CFPMWAg7tZglNVSQ', function=Function(arguments='{"size": 100, "asset": "XRP", "tp": 10}', name='take_profit'), type='function')])

#### **Third and best method for interpreting trading commands including time and executing their corresponding API calls**
(getting the list of all required times, given the list of all required times):




#### First example:

In [ ]:
from openai import OpenAI

# Initialize the OpenAI client with your API key
client = OpenAI(api_key='OPENAI_API_KEY')

# Define the user prompt
user_prompt = """
"At 2:00 pm, buy 50 BTC but right now buy 2 ETH."
"""

# Define the final message content
final_msg_content = f"""
 list the exact times for each action in the format:
- Current time: [Current Time]
- Time to do name the action

{user_prompt}
"""

# Define the messages for the OpenAI model
messages = [
    {
        "role": "system",
        "content": "Pretend you are an expert trader with knowledge in coding and list the times for each action in the specified format without any further explanation."
    },
    {"role": "user", "content": final_msg_content}
]

# Get the completion from OpenAI model
completion = client.chat.completions.create(
    model="gpt-4",
    messages=messages,
    temperature=0,
)

pprint(completion.choices[0].message.content)

('- Current time: [Current Time]\n'
 '- Time to buy 2 ETH: [Current Time]\n'
 '- Time to buy 50 BTC: 2:00 pm')


In [ ]:
# Initialize the OpenAI client with your API key
client = OpenAI(api_key='OPENAI_API_KEY')

# Define the user prompt
user_prompt = "At 2:00 pm, buy 50 BTC but right now buy 2 ETH."

# Update the final message content for a specific timing response
final_msg_content = f"""
Given the following time,
- current time : 1:00 pm
- Time to buy 2 ETH: 1:00 pm
- Time to buy 50 BTC: 2:00 pm
 Convert the following prompt into a concise, single-line, comma-separated list of actions and their specific timing in the following format: 'first determine the action then its corresponded specified time just in mumber'. Do not include any explanations or additional text.

{user_prompt}
"""

# Define the messages for the OpenAI model
messages = [
    {"role": "system", "content": "You are an expert trader and need to format the commands in the form 'first determine the action then its corresponded specified time' for each action. Do not include any explanations or additional text."},
    {"role": "user", "content": final_msg_content}
]

messages = [
    {"role": "system", "content": "You are an expert trader and need to format the commands as a single-line list with each command separated by a comma in the form 'first determine the action then its corresponded specified time just in' for each action. Do not include any explanations or additional text."},
    {"role": "user", "content": final_msg_content}
]


# Get the completion from OpenAI model
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    temperature=0,
)

# Print the response content
response = completion.choices[0].message.content

# Format the response to ensure it meets the exact requirements
formatted_response = response.strip()
formatted_response = formatted_response.replace('\n', ', ')

# Add a list numbering prefix
formatted_response = f'1. {formatted_response}'

# Print the formatted response
pprint(formatted_response)

'1. buy 2 ETH 1:00 pm, buy 50 BTC 2:00 pm'


In [ ]:
from openai import OpenAI

# Initialize the OpenAI client with your API key
client = OpenAI(api_key='OPENAI_API_KEY')

# Define the user prompt
user_prompt = """
'1. buy 2 ETH 1:00 pm, buy 50 BTC 2:00 pm'
"""

# Define the final message content
final_msg_content = f"""
Given the following times:
- current time: 12:00 pm
- Time to buy 2 ETH: 1:00 pm
- Time to buy 50 BTC: 2:00 pm
Check if it is the exact time to perform each action. If it is, execute the action and output the exact time of execution. If it is not the time, inform that it is not time yet and state when the action will be executed.

{user_prompt}
"""

# Define the messages for the OpenAI model
messages = [
    {
        "role": "system",
        "content": """
        You are a trading assistant. Check if it is the exact time to perform each trading command.
        If it is the exact time, execute the action and output the exact time of execution in the format: "[name the action] executed at [Time]".
        If it is not the time, inform that it is not time yet and state when the action should be executed in the format: "It's not time to [name the action]. It should be executed at [Time].".
        """
    },
    {"role": "user", "content": final_msg_content}
]

# Get the completion from OpenAI model
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    tool_choice="auto",
    temperature=0,
)

# Get the completion from OpenAI model
pprint(completion.choices)

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="It's not time to buy 2 ETH. It should be executed at 1:00 pm.\nIt's not time to buy 50 BTC. It should be executed at 2:00 pm.", role='assistant', function_call=None, tool_calls=None))]


In [ ]:
from openai import OpenAI

# Initialize the OpenAI client with your API key
client = OpenAI(api_key='OPENAI_API_KEY')

# Define the user prompt
user_prompt = """
'1. buy 2 ETH 1:00 pm, buy 50 BTC 2:00 pm'
"""

# Define the final message content
final_msg_content = f"""
Given the following times:
- current time: 1:00 pm
- Time to buy 2 ETH: 1:00 pm
- Time to buy 50 BTC: 2:00 pm
Check if it is the exact time to perform each action. If it is, execute the action and output the exact time of execution. If it is not the time, inform that it is not time yet and state when the action will be executed.

{user_prompt}
"""

# Define the messages for the OpenAI model
messages = [
    {
        "role": "system",
        "content": """
        You are a trading assistant. Check if it is the exact time to perform each trading command.
        If it is the exact time, execute the action and output the exact time of execution in the format: "[name the action] executed at [Time]".
        If it is not the time, inform that it is not time yet and state when the action should be executed in the format: "It's not time to [name the action]. It should be executed at [Time].".
        """
    },
    {"role": "user", "content": final_msg_content}
]

# Get the completion from OpenAI model
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    tool_choice="auto",
    temperature=0,
)

# Get the completion from OpenAI model
pprint(completion.choices)

[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='### Checking the Actions\n\n#### Action 1: Buy 2 ETH\n- **Scheduled Time:** 1:00 pm\n- **Current Time:** 1:00 pm\n\nSince it is the exact time to buy 2 ETH, I will execute this action.\n\n#### Action 2: Buy 50 BTC\n- **Scheduled Time:** 2:00 pm\n- **Current Time:** 1:00 pm\n\nIt is not the time to buy 50 BTC yet. This action should be executed at 2:00 pm.\n\n### Executing the Action\n\nI will now proceed to buy 2 ETH.\n\n', role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_RVs23AuslTkgYMvDCrkN1Ul1', function=Function(arguments='{"size":2,"asset":"ETH"}', name='buy'), type='function')]))]


In [ ]:
from openai import OpenAI

# Initialize the OpenAI client with your API key
client = OpenAI(api_key='OPENAI_API_KEY')

# Define the user prompt
user_prompt = """
'1. buy 2 ETH 1:00 pm, buy 50 BTC 2:00 pm'
"""

# Define the final message content
final_msg_content = f"""
Given the following times:
- current time: 1:30 pm
- Time to buy 2 ETH: 1:00 pm
- Time to buy 50 BTC: 2:00 pm
Check if it is the exact time to perform each action. If it is, execute the action and output the exact time of execution. If it is not the time, inform that it is not time yet and state when the action will be executed.

{user_prompt}
"""

# Define the messages for the OpenAI model
messages = [
    {
        "role": "system",
        "content": """
        You are a trading assistant. Check if it is the exact time to perform each trading command.
        If it is the exact time, execute the action and output the exact time of execution in the format: "[name the action] executed at [Time]".
        If it is not the time, inform that it is not time yet and state when the action should be executed in the format: "It's not time to [name the action]. It should be executed at [Time].".
        """
    },
    {"role": "user", "content": final_msg_content}
]

# Get the completion from OpenAI model
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    tool_choice="auto",
    temperature=0,
)

# Get the completion from OpenAI model
pprint(completion.choices)

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="- It's not time to buy 2 ETH. It should be executed at 1:00 pm.\n- It's not time to buy 50 BTC. It should be executed at 2:00 pm.", role='assistant', function_call=None, tool_calls=None))]


In [ ]:
from openai import OpenAI

# Initialize the OpenAI client with your API key
client = OpenAI(api_key='OPENAI_API_KEY')

# Define the user prompt
user_prompt = """
'1. buy 2 ETH 1:00 pm, buy 50 BTC 2:00 pm'
"""

# Define the final message content
final_msg_content = f"""
Given the following times:
- current time: 2:00 pm
- Time to buy 2 ETH: 1:00 pm
- Time to buy 50 BTC: 2:00 pm
Check if it is the exact time to perform each action. If it is, execute the action and output the exact time of execution. If it is not the time, inform that it is not time yet and state when the action will be executed.

{user_prompt}
"""

# Define the messages for the OpenAI model
messages = [
    {
        "role": "system",
        "content": """
        You are a trading assistant. Check if it is the exact time to perform each trading command.
        If it is the exact time, execute the action and output the exact time of execution in the format: "[name the action] executed at [Time]".
        If it is not the time, inform that it is not time yet and state when the action should be executed in the format: "It's not time to [name the action]. It should be executed at [Time].".
        """
    },
    {"role": "user", "content": final_msg_content}
]

# Get the completion from OpenAI model
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    tool_choice="auto",
    temperature=0,
)

# Get the completion from OpenAI model
pprint(completion.choices)

[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content="- It's not time to buy 2 ETH. It should be executed at 1:00 pm.\n- buy 50 BTC executed at 2:00 pm.\n\nExecuting the buy order for 50 BTC now.", role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_jBV0sGMmpwfjmqT2VGil4IaQ', function=Function(arguments='{"size":50,"asset":"BTC"}', name='buy'), type='function')]))]


#### Second example:

In [ ]:
from openai import OpenAI

# Initialize the OpenAI client with your API key
client = OpenAI(api_key='OPENAI_API_KEY')

# Define the user prompt
user_prompt = """
"30 minutes from now buy 2 BTC, then fifteen minutes after buy 3 ETH and buy 1 SOL an hour from now."
"""

# Define the final message content
final_msg_content = f"""
 list the exact times for each action in the format:
- Current time: [Current Time]
- Time to do name the action

{user_prompt}
"""

# Define the messages for the OpenAI model
messages = [
    {
        "role": "system",
        "content": "Pretend you are an expert trader with knowledge in coding and list the times for each action in the specified format without any further explanation."
    },
    {"role": "user", "content": final_msg_content}
]

# Get the completion from OpenAI model
completion = client.chat.completions.create(
    model="gpt-4",
    messages=messages,
    temperature=0,
)

pprint(completion.choices[0].message.content)


('- Current time: [Current Time]\n'
 '- Time to buy 2 BTC: [Current Time + 30 minutes]\n'
 '- Time to buy 3 ETH: [Current Time + 45 minutes]\n'
 '- Time to buy 1 SOL: [Current Time + 1 hour]')


In [ ]:
# Initialize the OpenAI client with your API key
client = OpenAI(api_key='OPENAI_API_KEY')

# Define the user prompt
user_prompt = "30 minutes from now buy 2 BTC, then fifteen minutes after buy 3 ETH and then buy 1 SOL an hour from now."

# Update the final message content for a specific timing response
final_msg_content = f"""
Given the following time,
- current time : 10:00 am
- Time to buy 2 BTC: 10:30 am
- Time to buy 3 ETH: 10:45 am
- Time to buy 1 SOL: 11:00 am
 Convert the following prompt into a concise, single-line, comma-separated list of actions and their specific timing in the following format: 'first determine the action then its corresponded specified time'. Do not include any explanations or additional text.

{user_prompt}
"""

# Define the messages for the OpenAI model
messages = [
    {"role": "system", "content": "You are an expert trader and need to format the commands in the form 'first determine the action then its corresponded specified time' for each action. Do not include any explanations or additional text."},
    {"role": "user", "content": final_msg_content}
]

messages = [
    {"role": "system", "content": "You are an expert trader and need to format the commands as a single-line list with each command separated by a comma in the form 'first determine the action then its corresponded specified time' for each action. Do not include any explanations or additional text."},
    {"role": "user", "content": final_msg_content}
]


# Get the completion from OpenAI model
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    temperature=0,
)

# Print the response content
response = completion.choices[0].message.content

# Format the response to ensure it meets the exact requirements
formatted_response = response.strip()
formatted_response = formatted_response.replace('\n', ', ')

# Add a list numbering prefix
formatted_response = f'1. {formatted_response}'

# Print the formatted response
pprint(formatted_response)

'1. buy 2 BTC at 10:30 am, buy 3 ETH at 10:45 am, buy 1 SOL at 11:00 am'


In [ ]:
# Initialize the OpenAI client with your API key
client = OpenAI(api_key='OPENAI_API_KEY')

# Define the user prompt
user_prompt = """
'1. buy 2 BTC at 10:30 am, buy 3 ETH at 10:45 am, buy 1 SOL at 11:00 am'
"""

# Define the final message content
final_msg_content = f"""
Given the following times:
- current time : 10:00 am
- Time to buy 2 BTC: 10:30 am
- Time to buy 3 ETH: 10:45 am
- Time to buy 1 SOL: 11:00 am
Check if it is the exact time to perform each action. If it is, execute the action and output the exact time of execution. If it is not the time, inform that it is not time yet and state when the action will be executed.

{user_prompt}
"""

# Define the messages for the OpenAI model
messages = [
    {
        "role": "system",
        "content": """
        You are a trading assistant. Check if it is the exact time to perform each trading command.
        If it is the exact time, execute the action and output the exact time of execution in the format: "[name the action] executed at [Time]".
        If it is not the time, inform that it is not time yet and state when the action should be executed in the format: "It's not time to [name the action]. It should be executed at [Time].".
        """
    },
    {"role": "user", "content": final_msg_content}
]

# Get the completion from OpenAI model
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    tool_choice="auto",
    temperature=0,
)

# Get the completion from OpenAI model
pprint(completion.choices)

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="- It's not time to buy 2 BTC. It should be executed at 10:30 am.\n- It's not time to buy 3 ETH. It should be executed at 10:45 am.\n- It's not time to buy 1 SOL. It should be executed at 11:00 am.", role='assistant', function_call=None, tool_calls=None))]


In [ ]:
# Initialize the OpenAI client with your API key
client = OpenAI(api_key='OPENAI_API_KEY')

# Define the user prompt
user_prompt = """
'1. buy 2 BTC at 10:30 am, buy 3 ETH at 10:45 am, buy 1 SOL at 11:00 am'
"""

# Define the final message content
final_msg_content = f"""
Given the following times:
- current time : 10:30 am
- Time to buy 2 BTC: 10:30 am
- Time to buy 3 ETH: 10:45 am
- Time to buy 1 SOL: 11:00 am
Check if it is the exact time to perform each action. If it is, execute the action and output the exact time of execution. If it is not the time, inform that it is not time yet and state when the action will be executed.

{user_prompt}
"""

# Define the messages for the OpenAI model
messages = [
    {
        "role": "system",
        "content": """
        You are a trading assistant. Check if it is the exact time to perform each trading command.
        If it is the exact time, execute the action and output the exact time of execution in the format: "[name the action] executed at [Time]".
        If it is not the time, inform that it is not time yet and state when the action should be executed in the format: "It's not time to [name the action]. It should be executed at [Time].".
        """
    },
    {"role": "user", "content": final_msg_content}
]

# Get the completion from OpenAI model
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    tool_choice="auto",
    temperature=0,
)

# Get the completion from OpenAI model
pprint(completion.choices)

[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content="### Checking Actions\n\n1. **Buy 2 BTC at 10:30 am**\n   - Current time: 10:30 am\n   - Scheduled time: 10:30 am\n   - Status: It is the exact time to perform this action.\n\n2. **Buy 3 ETH at 10:45 am**\n   - Current time: 10:30 am\n   - Scheduled time: 10:45 am\n   - Status: It's not time to perform this action. It should be executed at 10:45 am.\n\n3. **Buy 1 SOL at 11:00 am**\n   - Current time: 10:30 am\n   - Scheduled time: 11:00 am\n   - Status: It's not time to perform this action. It should be executed at 11:00 am.\n\n### Executing Actions\n\nSince it is the exact time to buy 2 BTC, I will proceed with this action.\n\n", role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_5xRqlkhJHlP6EeSgSX92XbTs', function=Function(arguments='{"size":2,"asset":"BTC"}', name='buy'), type='function')]))]


In [ ]:
# Initialize the OpenAI client with your API key
client = OpenAI(api_key='OPENAI_API_KEY')

# Define the user prompt
user_prompt = """
'1. buy 2 BTC at 10:30 am, buy 3 ETH at 10:45 am, buy 1 SOL at 11:00 am'
"""

# Define the final message content
final_msg_content = f"""
Given the following times:
- current time : 10:45 am
- Time to buy 2 BTC: 10:30 am
- Time to buy 3 ETH: 10:45 am
- Time to buy 1 SOL: 11:00 am
Check if it is the exact time to perform each action. If it is, execute the action and output the exact time of execution. If it is not the time, inform that it is not time yet and state when the action will be executed.

{user_prompt}
"""

# Define the messages for the OpenAI model
messages = [
    {
        "role": "system",
        "content": """
        You are a trading assistant. Check if it is the exact time to perform each trading command.
        If it is the exact time, execute the action and output the exact time of execution in the format: "[name the action] executed at [Time]".
        If it is not the time, inform that it is not time yet and state when the action should be executed in the format: "It's not time to [name the action]. It should be executed at [Time].".
        """
    },
    {"role": "user", "content": final_msg_content}
]

# Get the completion from OpenAI model
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    tool_choice="auto",
    temperature=0,
)

# Get the completion from OpenAI model
pprint(completion.choices)

[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content="1. **Buy 2 BTC at 10:30 am**:\n   - It's not time to buy 2 BTC. It should be executed at 10:30 am.\n\n2. **Buy 3 ETH at 10:45 am**:\n   - Buy 3 ETH executed at 10:45 am.\n\n3. **Buy 1 SOL at 11:00 am**:\n   - It's not time to buy 1 SOL. It should be executed at 11:00 am.\n\nI will now execute the action to buy 3 ETH.\n\nExecuting the buy order for 3 ETH...", role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_W9hEuggx8PZAVUfQWVdm23BS', function=Function(arguments='{"size":3,"asset":"ETH"}', name='buy'), type='function')]))]


In [ ]:
# Initialize the OpenAI client with your API key
client = OpenAI(api_key='OPENAI_API_KEY')

# Define the user prompt
user_prompt = """
'1. buy 2 BTC at 10:30 am, buy 3 ETH at 10:45 am, buy 1 SOL at 11:00 am'
"""

# Define the final message content
final_msg_content = f"""
Given the following times:
- current time : 10:50 am
- Time to buy 2 BTC: 10:30 am
- Time to buy 3 ETH: 10:45 am
- Time to buy 1 SOL: 11:00 am
Check if it is the exact time to perform each action. If it is, execute the action and output the exact time of execution. If it is not the time, inform that it is not time yet and state when the action will be executed.

{user_prompt}
"""

# Define the messages for the OpenAI model
messages = [
    {
        "role": "system",
        "content": """
        You are a trading assistant. Check if it is the exact time to perform each trading command.
        If it is the exact time, execute the action and output the exact time of execution in the format: "[name the action] executed at [Time]".
        If it is not the time, inform that it is not time yet and state when the action should be executed in the format: "It's not time to [name the action]. It should be executed at [Time].".
        """
    },
    {"role": "user", "content": final_msg_content}
]

# Get the completion from OpenAI model
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    tool_choice="auto",
    temperature=0,
)

pprint(completion.choices)

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="1. **Buy 2 BTC at 10:30 am**:\n   - It's not time to buy 2 BTC. It should be executed at 10:30 am.\n\n2. **Buy 3 ETH at 10:45 am**:\n   - It's not time to buy 3 ETH. It should be executed at 10:45 am.\n\n3. **Buy 1 SOL at 11:00 am**:\n   - It's not time to buy 1 SOL. It should be executed at 11:00 am.", role='assistant', function_call=None, tool_calls=None))]
